# Satellite Acquisition

**BEFORE RUNNING THIS CODE:
test.bin needs to be generated using gps-sdr-sim, using the command below. https://github.com/osqzss/gps-sdr-sim The brdc2000.15n file is from: ftp://cddis.gsfc.nasa.gov/gnss/data/daily/

This notebook demonstrates the satellite acquisition proccess, in which we determine which satellites are overhead. This code implements the parallel code phase search algorithm, which is the most complex of those available, but is also the fastest since it returns a result in just one iteration. It is possible that we will need to use a different algorithm when the code is running on an embedded platform.

As noted in the L-12 GPS data README, the sampling frequency is 4.092 MHz. and the IF is 0. Also note that the L1 signal hits the antenna at the 1.57542 GHz frequency (10.23 MHz x 154). This is dealt with by the hardware frontend, so we only have to concern ourselves with the 1.023 MHz BPSK modulated C/A code.

## Preamble

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import struct
import math
import heapq

##Grabs the GoldCode module from the root of the repo
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from GoldCode import GoldCode
from GPSData import IQData

## Import data File

test.bin needs to be generated using gps-sdr-sim, using the command below. 
https://github.com/osqzss/gps-sdr-sim
The brdc2000.15n file is from: 
ftp://cddis.gsfc.nasa.gov/gnss/data/daily/

In [13]:
# Generated from gps-sdr-sim using the command: 
#  ./gps-sdr-sim -e brdc2000.15n -l 43.798582,-120.648512,1381.0 -t 2015/07/19,11:17:11 -s 5000000 -d 5 -b 8 -o test5MHz.bin -v

fs = 5*10**6 # Sample rate used in above command

fHandle = open('../../gps-sdr-sim/test5MHz.bin','rb')

# Read first pair of bytes
I1 = fHandle.read(1)
Q1 = fHandle.read(1)

i = 1
numOfMS = 5
ms = 1*10**(-3)
TotalSamples = (numOfMS*ms) * fs
SamplesInOneMS = ms*fs

# Create empty lists
IData = []
QData = []

# Read until no data is left (will also stop if goes past desired numOfMS)
while I1 != "":
    IData.append(int.from_bytes(I1, byteorder='big', signed=True))
    QData.append(int.from_bytes(Q1, byteorder='big', signed=True))
    I1 = fHandle.read(1)
    Q1 = fHandle.read(1)
    i = i+1
    if i > TotalSamples: # Read only desired number of ms
        break
fHandle.close()

# Generate time vector
t = np.linspace(0,(numOfMS*ms),len(IData),endpoint=True) 

# Create Numpy array of complex data (Note: IData is the real data)
CData = np.zeros(len(IData), dtype=np.complex)
CData = np.array(IData) + 1j*np.array(QData)

In [14]:
# Since we are using circular-convolution, nfft must be equal to the number of samples
nfft = len(CData)
deltaFreq = fs/nfft
print("Calculated deltaFreq: %f [Hz]" %(deltaFreq))

# Generate frequency lists for plotting fft
# Can use f for regular fft and fshift for shifted version
fs_kHz = fs/1000 #Make easier to read
f = np.linspace (0,fs_kHz,nfft, endpoint=False)
fshift = np.linspace(-fs_kHz/2,fs_kHz/2,nfft,endpoint=False)

Calculated deltaFreq: 200.000000 [Hz]


## Function to generate gold codes

In [15]:
def GoldCodeSampled(PRN,fs,ChipOffset,numOfMS = 1):
    # Create list of C/A code Taps, for simpler sat selection",
    sat = [(1,5),(2,6),(3,7),(4,8),(0,8),(1,5),(0,7),(1,8),(2,9),(1,2),(2,3),(4,5),(5,6),(6,7),(7,8),(8,9),(0,3),(1,4),(2,5),(3,6),(4,7),(5,8),(0,2),(3,5),(4,6),(5,7),(6,8),(7,9),(0,5),(1,6),(2,7),(3,8),(4,9),(3,9),(0,6),(1,7),(3,9)]
    
    # Create Code Generator object for chosen Satellite
    CodeGen = GoldCode(sat[PRN - 1]) # Index starts at zero
    
    # Generate a non-sampled CA Code
    CACode = CodeGen.getCode(1023*numOfMS, samplesPerChip = 1)
    
    # Determine how many samples occur within numOfMS
    numOfSamples = (numOfMS*10**(-3)*fs)
    maxTimeValue = numOfMS*10**(-3)
    
    # Determine ChipOffset in units of time
    Tca = 1/(1.023*10**6)
    timeChipOffset = Tca*ChipOffset 
    
    # Time vector for sampled CA code
    tm = np.linspace(timeChipOffset, maxTimeValue + timeChipOffset, endpoint=True)

    # Time vector for 1.023 MHz sampled CA code (1 sample per chip)
    # where each sample is at the beginning transition edge (ie: the first chip starts at t=0)
    tca = np.linspace(0,maxTimeValue,numOfMS*1023,endpoint=False)
    
    CACodeSampled = []
    for tInd,tVal in enumerate(tm):
        # Catch values outside the boundry of 0 [ms] - n [ms] CA code
        # Then roll the time to the other end of the time list and store in tAdj
        if tVal < 0: 
            tAdj = maxTimeValue + tVal
        elif tVal > maxTimeValue:
            tAdj = maxTimeValue - tVal
        else: 
            tAdj = tVal
        
    return 0 # Haven't finished implementing this part yet.

In [16]:
CodeSampled = GoldCodeSampled(1,fs,0,1)


## Function to find second largest value

In [17]:
# This will return second largest value
# It will also ignore any value that is close to the second largest value
def GetSecondLargest(DataList):
    # Make sure is a numpy array
    DataArray = np.array(DataList)
    
    # Find largest value
    Largest = np.amax(DataArray)
    LargestIndex = np.argmax(DataArray)
#    print("Largest value: %f, at position: %d"%(Largest,LargestIndex))
    
    # Reduce value by a percent to prevent near-identical values from being selected
    ScaleAmount = 0.95
    ScaledLargest = ScaleAmount*Largest
    SecondLargest = 0
    SecondLargestIndex = 0
    
    for ind,val in enumerate(DataArray):
        if val < ScaledLargest:
            if val > SecondLargest:
                #Ignore adjacent bins to Largest
                if (np.abs(LargestIndex-ind) > 10):
                    SecondLargest = val
                    SecondLargestIndex = ind
    
#    print("Second largest value: %f, at position: %d"%(SecondLargest,SecondLargestIndex))
    return SecondLargest